#Taking outputs from ABodyBuilder and MAPT, and embedding associated antibody sequences in Ankh and Ablang to obtain amino acid sequence representations.

Next steps are to run the per residue embeddings for the ankh scfv sequences.

Need to work out the max seq length

Need to save the embeddings to disk

Need to work out how to group the batches together

Need to assess whether features from per residue embedding are more predictive of MAPT score than per sequence embedding.

How will this be done? How can a 2D array be used to train a neural network?


In [16]:
#@title Mount drive and load libraries
%%capture
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install ankh

import os, gc
import pandas as pd, numpy as np
from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc-project-mbalmf01/msc-project-source-code-files-22-23-mbalmf01/notebooks'
os.chdir(path)

import torch

In [14]:
df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230813_human_paired_seqs_MAPT_scores.csv', dtype={'Run':np.str_})
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'sequence_id_heavy', 'ANARCI_status_heavy',
       'sequence_heavy', 'sequence_alignment_aa_heavy', 'v_call_heavy',
       'd_call_heavy', 'j_call_heavy', 'sequence_id_light',
       'ANARCI_status_light', 'sequence_light', 'v_call_light', 'j_call_light',
       'sequence_alignment_aa_light', 'Run', 'seq_id', 'scfv', 'Acidics Fv',
       'Basics Fv', 'Charge pH5 Fv', 'Charge pH5 H', 'Charge pH5 L',
       'Charge pH7 Fv', 'Charge pH7 H', 'Charge pH7 L', 'Filename',
       'Hydrophobic Surface Fv', 'Hydrophobic Surface H',
       'Hydrophobic Surface L', 'MAPSS IgG1 pH5.0', 'MAPSS IgG1 pH7.4',
       'MAPSS IgG4P pH5.0', 'MAPSS IgG4P pH7.4', 'Model Seq L'],
      dtype='object')

In [18]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('qilowoq/AbLang_light')
model = AutoModel.from_pretrained('qilowoq/AbLang_light', trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/qilowoq/AbLang_light:
- AbLang_roberta_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
def get_aa_embedding(sequence: str, model: str, tokenizer, max_length: int):
  sequence = ' '.join(sequence)
  encoded_input = tokenizer(sequence, padding='max_length', return_tensors='pt')
  model_output = model(**encoded_input)
  return model_output.last_hidden_state

In [5]:
device='cpu'
def aa_embeddings(seqs: list, model, tokenizer, max_length: int, device) -> list:
    seqs = [" ".join(list(sequence)) for sequence in seqs]
    ids = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)
    ember = embedding_repr.last_hidden_state
    return ember

In [ ]:
import gc

seqs = df['sequence_alignment_aa_heavy'].to_list()
max_length = max([len(seq) for seq in seqs])

def chunks(l, n):
    """Splits a list into evenly sized chunks."""
    return [l[i:i + n] for i in range(0, len(l), n)]

seqs_chunked = chunks(seqs, len(seqs) // 500)

for num, seq in enumerate(seqs_chunked):
  embeddings = [get_aa_embedding(s, model, tokenizer, max_length) for s in seq]
  embeddings = torch.cat(embeddings, dim=0)
  filepath = f"/content/drive/MyDrive/msc-project-mbalmf01/embeddings/230814_ablang_H_aa_embeddings_{num}.pt"
  torch.save(embeddings, filepath)
  del embeddings
  gc.collect()

In [ ]:
l = []
for j in os.listdir('/content/drive/MyDrive/msc-project-mbalmf01/embeddings/'):
  t = torch.load(f'/content/drive/MyDrive/msc-project-mbalmf01/embeddings/{j}')
  t = t.detach().numpy()
  l.append(t)

In [5]:
ten = np.concatenate(l)

In [11]:
import gc
del ten
gc.collect()

68

In [10]:
ten.shape

(29346, 160, 768)

In [7]:
np.save('/content/array2.npy', ten)

In [12]:
file_size = os.path.getsize('/content/array2.npy')
print(f'file_size is {round(file_size/1073741824, 3)}GB')

file_size is 13.434GB


In [13]:
!cp /content/array2.npy /content/drive/MyDrive/msc-project-mbalmf01/embeddings/array.npy

In [20]:
#@title Repeat for light chain embeddings
def get_aa_embedding(sequence: str, model: str, tokenizer, max_length: int):
  sequence = ' '.join(sequence)
  encoded_input = tokenizer(sequence, padding='max_length', return_tensors='pt')
  model_output = model(**encoded_input)
  lhs = model_output.last_hidden_state
  return lhs.detach().numpy()

seqs = df['sequence_alignment_aa_light'].to_list()
max_length = max([len(seq) for seq in seqs])

seqs_chunked = chunks(seqs, len(seqs) // 10)

for num, seq in enumerate(seqs_chunked):
  embeddings = [get_aa_embedding(s, model, tokenizer, max_length) for s in seq]
  embeddings = np.concatenate(embeddings)
  filepath = f"/content/230814_ablang_L_aa_embeddings_{num}.npy"
  np.save(filepath, embeddings)
  del embeddings
  gc.collect()

In [21]:
import shutil
for i in os.listdir('/content/'):
  if '230814' in i:
    shutil.copy(f'/content/{i}',
                f'/content/drive/MyDrive/msc-project-mbalmf01/embeddings/{i}')

In [ ]:
from google.colab import runtime

def disconnect_and_delete_runtime():
  """Disconnects and deletes the current runtime."""
  runtime.unassign()

disconnect_and_delete_runtime()